# Acquisition and Prep Exercises II

Let's set up an example scenario as perspective for our regression exercises using the Telco dataset.

As a customer analyst for Telco, you want to know who has spent the most money with the company over their lifetime. You have monthly charges and tenure, so you think you will be able to use those two attributes as features to estimate total charges. You need to do this within an average of $5.00 per customer.

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Telco data from the telco_churn database in the Codeup database server.

1. Acquire customer_id, monthly_charges, tenure, and total_charges from the telco_churn database for all customers with a 2-year contract.

In [1]:
import pandas as pd
import numpy as np
import os
from env import host, username, password

In [2]:
def get_db_url(db, username=username, host=host, password=password):
    
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'

In [3]:
def new_telco_data():
    '''
    gets telco_churn information from CodeUp db and creates a dataframe
    '''

    # SQL query
    telco_query = '''SELECT customer_id, monthly_charges, tenure, total_charges
                     FROM customers
                     WHERE contract_type_id = 3'''
    
    # reads SQL query into a DataFrame            
    df = pd.read_sql(telco_query, get_db_url('telco_churn'))
    
    return df

In [4]:
def get_telco_data():
    '''
    checks for existing telco_churn csv file and loads if present,
    otherwise runs new_telco_data function to acquire data
    '''
    
    # checks for existing file and loads
    if os.path.isfile('telco_churn.csv'):
        
        df = pd.read_csv('telco_churn.csv', index_col=0)
        
    else:
        
        # pull in data and creates csv file if not already present
        df = new_telco_data()
        
        df.to_csv('telco_churn.csv')
    
    return df

In [5]:
telcodf = get_telco_data()
telcodf.head()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.8
2,0016-QLJIS,90.45,65,5957.9
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


2. Using your acquired Telco data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [6]:
telcodf.shape

(1695, 4)

In [7]:
telcodf.describe().T

,count,mean,std,min,25%,50%,75%,max
monthly_charges,1695.0,60.770413,34.678865,18.4,24.025,64.35,90.45,118.75
tenure,1695.0,56.735103,18.209363,0.0,48.000,64.00,71.00,72.00


In [8]:
telcodf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 66.2+ KB


- monthly_charges is float, might contain nulls
- total_charges is object, investigate

In [9]:
telcodf.isnull().sum()

customer_id        0
monthly_charges    0
tenure             0
total_charges      0
dtype: int64

In [10]:
telcodf.monthly_charges.value_counts(dropna=False, ascending=True)

95.75      1
29.60      1
63.70      1
88.80      1
114.35     1
          ..
19.75     17
19.55     18
20.05     19
19.85     21
19.70     21
Name: monthly_charges, Length: 820, dtype: int64

In [11]:
telcodf.total_charges.value_counts(dropna=False, ascending=False)

           10
3533.6      2
7334.05     2
343.45      2
1161.75     2
           ..
8337.45     1
1397.65     1
7657.4      1
1070.25     1
6302.85     1
Name: total_charges, Length: 1678, dtype: int64

In [12]:
# replace symbols, etc with NaN's
telcodf.replace(r'^\s*$', np.nan, regex=True, inplace=True)
telcodf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1685 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 66.2+ KB


In [13]:
# replace NaN's with monthly_charges
telcodf.total_charges = telcodf.total_charges.fillna(telcodf.monthly_charges)
telcodf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 66.2+ KB


In [14]:
# change total_charges data type to float
telcodf = telcodf.astype({'total_charges': 'float64'})
telcodf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 66.2+ KB


3. End with a wrangle.py file that contains the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values. Name your final function wrangle_telco.

In [15]:
from wrangle import wrangle_telco

In [16]:
telcodf = wrangle_telco()
telcodf.head()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.80
2,0016-QLJIS,90.45,65,5957.90
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


## Exercises III - Challenge

Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [17]:
def new_zillow_data():
    '''
    gets zillow information from CodeUp db and creates a dataframe
    '''

    # SQL query
    zillow_query = '''SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
                      FROM properties_2017
                      WHERE propertylandusetypeid = 261'''
    
    # reads SQL query into a DataFrame            
    df = pd.read_sql(zillow_query, get_db_url('zillow'))
    
    return df

In [18]:
def get_zillow_data():
    '''
    checks for existing zillow csv file and loads if present,
    otherwise runs new_zillow_data function to acquire data
    '''
    
    # checks for existing file and loads
    if os.path.isfile('zillow.csv'):
        
        df = pd.read_csv('zillow.csv')
        
    else:
        
        # pull in data and creates csv file if not already present
        df = new_zillow_data()
        
        df.to_csv('zillow.csv')
    
    return df

In [19]:
zillowdf = get_zillow_data()
zillowdf.head()

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.

In [20]:
zillowdf.shape

(2152863, 8)

In [21]:
zillowdf.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,2152863.0,1.076431e+06,621478.160628,0.00,538215.50,1076431.00,1614646.50,2152862.00
bedroomcnt,2152852.0,3.287196e+00,0.954754,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2152852.0,2.230688e+00,0.999280,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2144379.0,1.862855e+03,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2152370.0,4.618962e+05,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
yearbuilt,2143526.0,1.960950e+03,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2148421.0,5.634866e+03,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
fips,2152863.0,6.048377e+03,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00


In [22]:
zillowdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 8 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Unnamed: 0                    int64  
 1   bedroomcnt                    float64
 2   bathroomcnt                   float64
 3   calculatedfinishedsquarefeet  float64
 4   taxvaluedollarcnt             float64
 5   yearbuilt                     float64
 6   taxamount                     float64
 7   fips                          float64
dtypes: float64(7), int64(1)
memory usage: 131.4 MB


In [23]:
zillowdf.columns

Index(['Unnamed: 0', 'bedroomcnt', 'bathroomcnt',
       'calculatedfinishedsquarefeet', 'taxvaluedollarcnt', 'yearbuilt',
       'taxamount', 'fips'],
      dtype='object')

In [24]:
# drop column 'Unnamed: 0'
zillowdf.drop(columns='Unnamed: 0', inplace=True)
zillowdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 115.0 MB


In [25]:
# replace symbols, etc with NaN's
zillowdf.replace(r'^\s*$', np.nan, regex=True, inplace=True)
zillowdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 115.0 MB


In [26]:
zillowdf.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

In [27]:
# calcualte percentage of nulls
9337 / 2152863 * 100

0.43370154069255684

In [28]:
# drop nulls
zillowdf = zillowdf.dropna()
zillowdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 130.6 MB


In [29]:
zillowdf.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0


In [30]:
# house with no bedrooms AND no bathrooms?
# find observations where bedroomcnt = 0 AND bathroomcnt = 0
zillowdf[['bedroomcnt', 'bathroomcnt']][(zillowdf.bedroomcnt == 0) & (zillowdf.bathroomcnt == 0)]

,bedroomcnt,bathroomcnt
11,0.0,0.0
14,0.0,0.0
15,0.0,0.0
866,0.0,0.0
2306,0.0,0.0
...,...,...
2144648,0.0,0.0
2146189,0.0,0.0
2146660,0.0,0.0
2149054,0.0,0.0


In [32]:
zillowdf[(zillowdf.bedroomcnt == 0) & (zillowdf.bathroomcnt == 0)]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
15,0.0,0.0,203.0,14166.0,1960.0,163.79,6037.0
866,0.0,0.0,220.0,13840.0,1988.0,327.80,6037.0
2306,0.0,0.0,1776.0,79574.0,1989.0,1137.90,6037.0
...,...,...,...,...,...,...,...
2144648,0.0,0.0,936.0,33811.0,1961.0,536.25,6037.0
2146189,0.0,0.0,330.0,16311.0,1925.0,297.36,6037.0
2146660,0.0,0.0,676.0,234580.0,1922.0,3013.57,6037.0
2149054,0.0,0.0,938.0,56733.0,1978.0,986.86,6037.0


In [33]:
# drop houses with no bedrooms and bathrooms
zillowdf = zillowdf.drop(list(zillowdf[(zillowdf.bedroomcnt == 0) & (zillowdf.bathroomcnt == 0)].index))

In [34]:
zillowdf[['bedroomcnt', 'bathroomcnt']][(zillowdf.bedroomcnt == 0) & (zillowdf.bathroomcnt == 0)]

,bedroomcnt,bathroomcnt


In [41]:
# 
zillowdf[zillowdf.calculatedfinishedsquarefeet < 250]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
39196,0.0,1.0,78.0,52991.0,2009.0,866.12,6037.0
58438,2.0,1.0,1.0,121376.0,1907.0,1996.35,6037.0
70053,4.0,0.0,31.0,405275.0,1938.0,4892.42,6037.0
70999,3.0,2.0,206.0,70130.0,1955.0,1132.52,6037.0
75264,0.0,1.0,216.0,6269144.0,2011.0,75214.96,6037.0
...,...,...,...,...,...,...,...
2014731,1.0,1.0,144.0,92916.0,1923.0,1202.57,6037.0
2017745,3.0,1.0,1.0,31800.0,1900.0,870.36,6037.0
2028617,3.0,2.0,152.0,86419.0,1939.0,1565.27,6037.0
2029164,0.0,1.0,180.0,31.0,1992.0,91.13,6037.0


3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your wrangle.py file. Name your final function wrangle_zillow.

In [30]:
from wrangle import wrangle_zillow

In [31]:
zillowdf = wrangle_zillow()
zillowdf.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0


In [32]:
zillowdf.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64